# Set up wind farm run

In [ ]:
# Add any possible locations of amr-wind-frontend here
amrwindfedirs = ['./submods/amr-wind-frontend/']
import sys, os, shutil
for x in amrwindfedirs: sys.path.insert(1, x)

# Load the libraries
import amrwind_frontend  as amrwind
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy             as np
import math
import pandas as pd
import postproamrwindsample as ppsample
import time
import utm
import pickle

# Also ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Make all plots inline 
%matplotlib inline
plt.style.use("project.mplstyle")

cwd              = os.getcwd()

In [ ]:
# Start the AMR-Wind case
case = amrwind.MyApp.init_nogui()

In [ ]:
# Location of precursor run with boundary data
precursordir  = '../02_precursor_shell/neutral_8.6at150.10dTInv_0.75z0_750zi_10x7x1km_feb3/'
precursorsetup= os.path.join(precursordir, 'setup_precursor_neutral.i')

# Location of farm run dir
farmrundir       = os.path.join(cwd, 'demo_case')
outputfile       = 'demo_case.inp'
noturboutputfile = 'demo_case_noturbs.inp'

verbose          = True

# Go to the run directory
os.chdir(cwd)
if not os.path.exists(farmrundir):
    os.makedirs(farmrundir)
os.chdir(farmrundir)

# Load the starting point
case.loadAMRWindInput(precursorsetup)

In [ ]:
tstart = 20000 #chk65000 16125
tstop  = 21800
textra = 0

In [ ]:
# Set the time constants
case.setAMRWindInput('time.stop_time', tstop+textra)
case.setAMRWindInput('time.fixed_dt', 0.02)
case.setAMRWindInput('time.max_step',  -1)

In [ ]:
# Set the restart point
chkint = 40000
chkdir = precursordir+f'/chk{chkint}/'
case.setAMRWindInput('restart_file', os.path.realpath(chkdir))

In [ ]:
# Set the boundary input file parameters
ablstatfile = precursordir+'/post_processing/abl_statistics60000.nc'
tavg        =  [tstart, tstop]
forcingdict = {'ablstatfile':ablstatfile, 'tavg':tavg}
inflowplanes= ['xlo'] #['ylo', 'xhi']
bndryfiles  = os.path.join(precursordir, 'bndry_file.nc')

In [ ]:
case.boundaryplane_restart(bndryfiles=bndryfiles, 
                           forcingdict=forcingdict, 
                           inflowplanes=inflowplanes,
                           verbose=True)

In [ ]:
# Add the turbine specifications flag
## Note that the options field can have things like:
##    ADparam_TwrAero:False ADparam_TwrShadow:0 FSTparam_TMax:181234.0
OFoptions=("ADparam_TwrAero:True ADparam_TwrShadow:0 FSTparam_CompHydro:0 FSTparam_CompSub:0 "
           "EDparam_YawDOF:False "
           "EDparam_PtfmSgDOF:False "
           "EDparam_PtfmSwDOF:False "
           "EDparam_PtfmHvDOF:False "
           "EDparam_PtfmRDOF:False "
           "EDparam_PtfmPDOF:False "
           "EDparam_PtfmYDOF:False "
           "AMRparam_Actuator_openfast_stop_time:2000 "
          )

#"EDparam_RotSpeed:5.00 "

## To specify changes to the OpenFAST model
turbinescsv="""
# CSV file should have columns with
# name, x, y, type, yaw, hubheight, options
T1A, 12, -2000, IEA15MW_ALM, 250.0, , {OFoptions}
T2A, 1862, -2000, IEA15MW_ALM, 250.0, , {OFoptions}
T3A, 3712, -2000, IEA15MW_ALM, 260.0, , {OFoptions}
T4A, 5562, -2000, IEA15MW_ALM, 270.0, , {OFoptions}
#T5A, 7412, -2000, IEA15MW_ALM, 270.0, , {OFoptions}
T1B, 12, 0, IEA15MW_ALM, 270.0, , {OFoptions}
T2B, 1862, 0, IEA15MW_ALM, 270.0, , {OFoptions}
T3B, 3712, 0, IEA15MW_ALM, 270.0, , {OFoptions}
T4B, 5562, 0, IEA15MW_ALM, 270.0, , {OFoptions}
#T5B, 7412, 0, IEA15MW_ALM, 270.0, , {OFoptions}
T1C, 12, 2000, IEA15MW_ALM, 290.0, , {OFoptions}
T2C, 1862, 2000, IEA15MW_ALM, 290.0, , {OFoptions}
T3C, 3712, 2000, IEA15MW_ALM, 280.0, , {OFoptions}
T4C, 5562, 2000, IEA15MW_ALM, 270.0, , {OFoptions}
#T5C, 7412, 2000, IEA15MW_ALM, 270.0, , {OFoptions}
""".format(OFoptions=OFoptions)
case.setAMRWindInput('turbines_csvtextbox',  turbinescsv)

In [ ]:
case.setAMRWindInput('turbines_createnewdomain', False)
case.setAMRWindInput('turbines_deleteprev', True)

case.turbines_createAllTurbines()

In [ ]:
# Preview the turbine layout
fig, ax = plt.subplots(figsize=(5,5), facecolor='w', dpi=150)
case.turbines_previewAllTurbines(ax=ax)

## Make refinement regions

In [ ]:
refinementcsv="""
# CSV file should have columns with
# level, upstream, downstream, lateral, below, above, options
level, upstream, downstream, lateral, below, above, options
0,     750,    8000,      3000,  150,  500,      center:specified units:meter centerx:0 centery:0 centerz:150 name:Farm
1,  1,    1,    1,  1,  1.2, orientation:nacdir
"""
case.setAMRWindInput('refine_csvtextbox', refinementcsv)
case.setAMRWindInput('refine_deleteprev', True)

In [ ]:
case.refine_createAllZones()
# Print out existing list of refinement zones
print(case.listboxpopupwindict['listboxtagging'].getitemlist())

In [ ]:
# Estimate mesh size
case.estimateMeshSize(verbose=False)

In [ ]:
# Plot the domain
fig, ax2 = plt.subplots(figsize=(5,5), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_chooseview']      = 'XY'
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.popup_storteddata['plotdomain']['plot_refineboxes']     = case.listboxpopupwindict['listboxtagging'].getitemlist()
case.popup_storteddata['plotdomain']['plot_windnortharrows'] = False
case.plotDomain(ax=ax2)
ax2.set_title("")
plt.xlabel(r"$x (\mathrm{m})$")
plt.ylabel(r"$y (\mathrm{m})$")
pickle.dump(fig, open('farm.pickle', 'wb'))

In [ ]:
# Plot the domain
fig, ax = plt.subplots(figsize=(8,6), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_chooseview']      = 'YZ'
case.popup_storteddata['plotdomain']['plot_refineboxes']     = case.listboxpopupwindict['listboxtagging'].getitemlist()
case.popup_storteddata['plotdomain']['plot_sampleprobes']    = [] #case.listboxpopupwindict['listboxsampling'].getitemlist()
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.plotDomain(ax=ax)

## Add sampling planes

In [ ]:
# Delete all old sampling planes from precursor
case.listboxpopupwindict['listboxsampling'].deleteall()
case.listboxpopupwindict['listboxpostprosetup'].deleteall()
print(case.listboxpopupwindict['listboxsampling'].getitemlist())
print(case.listboxpopupwindict['listboxpostprosetup'].getitemlist())

In [ ]:
# Set up averaging
pprosetup = case.get_default_postprosetupdict()
pprosetup['postprocessing_setup_name'] = 'sampling_'
pprosetup['postprocessing_setup_type'] = 'Sampling'
pprosetup['postprocessing_setup_output_frequency'] =  1000
pprosetup['postprocessing_setup_fields']           =  ['velocity', 'temperature','tke']
case.add_postprosetup(pprosetup, verbose=True)
sampleplane = case.get_default_samplingdict()
# Modify the geometry
sampleplane['sampling_name']         = 'XYslice'
sampleplane['sampling_outputto']     = 'sampling_'
sampleplane['sampling_type']         = 'PlaneSampler'
sampleplane['sampling_p_num_points'] = [1024, 768]
sampleplane['sampling_p_origin']     = [-1095., -3845., 0]
sampleplane['sampling_p_axis1']      = [1095+9145., 0, 0]
sampleplane['sampling_p_axis2']      = [0, 3845+3845, 0]
sampleplane['sampling_p_normal']     = [0, 0, 1]
sampleplane['sampling_p_offsets']    = '150.0'
case.add_sampling(sampleplane)

pprosetup = case.get_default_postprosetupdict()
pprosetup['postprocessing_setup_name'] = 'samplingxz_'
pprosetup['postprocessing_setup_type'] = 'Sampling'
pprosetup['postprocessing_setup_output_frequency'] =  1000
pprosetup['postprocessing_setup_fields']           =  ['velocity', 'temperature','tke']
case.add_postprosetup(pprosetup, verbose=True)
sampleplaneA = case.get_default_samplingdict()
sampleplaneA['sampling_name']         = 'XZslice'
sampleplaneA['sampling_outputto']     = 'samplingxz_'
sampleplaneA['sampling_type']         = 'PlaneSampler'
sampleplaneA['sampling_p_num_points'] = [1024, 96]
sampleplaneA['sampling_p_origin']     = [-1095., 0., 0]
sampleplaneA['sampling_p_axis1']      = [1095+9145., 0, 0]
sampleplaneA['sampling_p_axis2']      = [0, 0, 960]
sampleplaneA['sampling_p_normal']     = [0, 1, 0]
sampleplaneA['sampling_p_offsets']    = '-2000 0.0 2000'
case.add_sampling(sampleplaneA)

In [ ]:
samplingcsv="""
# CSV file should have columns withturbinescsv=
# name, type, upstream, downstream, lateral, below, above, n1, n2, options
name, type, upstream, downstream, lateral, below, above, n1, n2, options
#cl1, centerline, 1,  0, none, none,  none,  11, 11, none
rp1, rotorplane, 2,  8, 2, 0.6,  1,  101, 101, noffsets:11 outputto:RP_ outputfreq:10
swA, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:275 applyto:T[0-2]A
swA, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:272.5 applyto:T3A
swA, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:270 applyto:T4A
swB, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:270 applyto:T[0-9]B
swC, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:265 applyto:T[0-2]C
swC, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:267.5 applyto:T3C
swC, streamwise, 2,  8, 0, 0.6,  1,  101, 101, outputto:SW_ outputfreq:10 orientation:270 applyto:T4C
#hh,  hubheight,2000,2000,960, 0,  none,  11, 11, usedx:10 units:meter center:farm orientation:x outputvars:velocity;tke;temperature outputfreq:10 outputto:hubheight_
"""

case.setAMRWindInput('sampling_csvtextbox', samplingcsv)
case.setAMRWindInput('sampling_deleteprev', False)

In [ ]:
case.sampling_createAllProbes(verbose=False)
# Print out existing list of turbines
print(case.listboxpopupwindict['listboxsampling'].getitemlist())

In [ ]:
# Plot the domain
fig, ax = plt.subplots(figsize=(8,6), facecolor='w', dpi=125)
case.popup_storteddata['plotdomain']['plot_chooseview']      = 'XY'
case.popup_storteddata['plotdomain']['plot_refineboxes']     = [] #case.listboxpopupwindict['listboxtagging'].getitemlist()
case.popup_storteddata['plotdomain']['plot_sampleprobes']    = case.listboxpopupwindict['listboxsampling'].getitemlist()
case.popup_storteddata['plotdomain']['plot_turbines']        = case.listboxpopupwindict['listboxactuator'].getitemlist()
case.plotDomain(ax=ax)

## Add some extra (manual) options

In [ ]:
case.extradictparams['TKE.interpolation'] = 'PiecewiseConstant'
case.extradictparams['nodal_proj.mg_rtol'] = 1.0e-6
case.extradictparams['nodal_proj.mg_atol'] = 1.0e-12
case.extradictparams['mac_proj.mg_rtol'] = 1.0e-6
case.extradictparams['mac_proj.mg_atol'] = 1.0e-12
case.extradictparams['diffusion.mg_rtol'] = 1.0e-6
case.extradictparams['diffusion.mg_atol'] = 1.0e-12
case.extradictparams['temperature_diffusion.mg_rtol'] = 1.0e-10
case.extradictparams['temperature_diffusion.mg_atol'] = 1.0e-13
case.extradictparams['io.outputs'] = 'actuator_src_term'
case.extradictparams['io.derived_outputs'] = 'q_criterion'
case.extradictparams['io.line_plot_int'] = 1
case.extradictparams['io.restart_file'] = chkdir
case.extradictparams['BoussinesqBuoyancy.read_temperature_profile'] = True
case.extradictparams['BoussinesqBuoyancy.tprofile_filename']        = 'avg_theta.dat'
case.extradictparams['ABL.wall_shear_stress_type'] = "local"
case.extradictparams['ABL.inflow_outflow_mode']    = True
case.extradictparams['ABL.wf_velocity']            = '2.500245 0.105173'
case.extradictparams['ABL.wf_vmag']                = 2.532626503500448
case.extradictparams['ABL.wf_theta']               = 300.0090871001242
case.extradictparams['ABL.reference_temperature'] = 300.0
case.extradictparams['ABL.temperature_heights'] = '0.0 700.0 800.0 1800.0'
case.extradictparams['ABL.temperature_values'] = '300.0 300.0 310.0 313.0'
case.extradictparams['ABL.perturb_temperature'] = False
case.extradictparams['ABL.cutoff_height'] = 50.0
case.extradictparams['ABL.perturb_velocity'] = True
case.extradictparams['ABL.perturb_ref_height'] = 50.0
case.extradictparams['ABL.Uperiods'] = 40.0
case.extradictparams['ABL.Vperiods'] = 25.0
case.extradictparams['ABL.deltaU'] = 0.25
case.extradictparams['ABL.deltaV'] = 0.25
case.extradictparams['ABL.kappa'] = .41
case.extradictparams['ABL.surface_roughness_z0'] = 0.75
case.extradictparams['ABL.normal_direction'] = 2
case.extradictparams['BodyForce.magnitude'] = '0.00045577915197378074 0.0009349589835232567 0.0'
case.extradictparams['averaging.averaging_window'] = 300.0
case.extradictparams['averaging.averaging_start_time'] = 20900.0
case.extradictparams['time.checkpoint_interval'] = 1000
case.extradictparams['time.plot_interval'] = 1000
case.extradictparams['time.checkpoint_start'] = chkint

## Print the input file

In [ ]:
# Write the input file
inputfile=case.writeAMRWindInput(outputfile)
if verbose: print(inputfile)

### Create a version without turbines

In [ ]:
def removeturbines(runcase):
    physics = runcase.getAMRWindInput('incflo.physics')
    if 'Actuator' in physics:
        physics.remove('Actuator')
        runcase.setAMRWindInput('physics', physics)
        print('SET incflo.physics: '+repr(runcase.getAMRWindInput('incflo.physics')))
    runcase.setAMRWindInput('ActuatorForcing', False)    
    runcase.listboxpopupwindict['listboxactuator'].deleteall()
    return

In [ ]:
#removeturbines(case)
#inputfile=case.writeAMRWindInput(noturboutputfile)
#if verbose: print(inputfile)